In [42]:
import pandas as pd
import random
import yahoo_fin.stock_info as si
import yfinance as yf
import json
import csv
import re
from io import StringIO
from bs4 import BeautifulSoup
import requests
import get_all_tickers.get_tickers as gt


In [8]:
url_shares_outstanding = "https://finance.yahoo.com/quote/{}/key-statistics?p={}"
# die {} dienen sozusagen als Platzhalter

In [9]:
tick = "F"

In [10]:
response = requests.get(url_shares_outstanding.format(tick, tick))

In [11]:
soup = BeautifulSoup(response.text, "html.parser")

In [12]:
pattern = re.compile(r"\s--\sData\s--\s")
script_data = soup.find("script", text = pattern).contents[0]

In [19]:
start = script_data.find("context")-2

In [21]:
json_data = json.loads(script_data[start:-12])

In [22]:
json_data["context"].keys()

dict_keys(['dispatcher', 'options', 'plugins'])

In [23]:
print(json_data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"].keys())


dict_keys(['defaultKeyStatistics', 'financialsTemplate', 'price', 'financialData', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'pageViews'])


In [39]:
shares_outstanding = json_data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["defaultKeyStatistics"]["sharesOutstanding"]["raw"]


In [43]:
all_tickers = gt.get_tickers()

6681


In [51]:
shares_tickers = []
shares_outstanding_list = []

for ticker in all_tickers[3000:]:
    try:
        response = requests.get(url_shares_outstanding.format(ticker, ticker))        
        soup = BeautifulSoup(response.text, "html.parser")
        pattern = re.compile(r"\s--\sData\s--\s")
        script_data = soup.find("script", text = pattern).contents[0]
        start = script_data.find("context")-2
        json_data = json.loads(script_data[start:-12])
        shares_outstanding = json_data["context"]["dispatcher"]["stores"]["QuoteSummaryStore"]["defaultKeyStatistics"]["sharesOutstanding"]["raw"]
        shares_tickers.append(ticker)
        shares_outstanding_list.append(shares_outstanding)
    except:
        pass
        


In [52]:
shares_out = pd.DataFrame({
    "ticker" : shares_tickers,
    "so" : shares_outstanding_list
})

In [53]:
shares_out.to_csv("shares_outstanding_ab_3000.csv")